In [ ]:
# If you don't have the required dependencies, uncomment and run the following lines:
# !pip install nltk numpy

import nltk
import math
import string
import random
import logging
import numpy as np

from nltk.util import ngrams
from collections import Counter
from nltk.tokenize import word_tokenize
from typing import List, Tuple, Dict, Set, Optional
from nltk.corpus import reuters, brown, gutenberg, webtext



In [2]:
alpha = 0.001

In [3]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = ''.join([char for char in text if not char.isdigit()])
    text = ' '.join(text.split())
    return text


def get_training_data() -> str:
    logging.getLogger("nltk").setLevel(logging.ERROR)
    nltk.download('reuters')
    nltk.download('brown')
    nltk.download('gutenberg')
    nltk.download('webtext')
    nltk.download('punkt')
    
    print("Gathering texts from multiple corpora...")
    
    reuters_text = ' '.join(reuters.raw(file_id) for file_id in reuters.fileids())
    brown_text = ' '.join(brown.raw(file_id) for file_id in brown.fileids())
    gutenberg_text = ' '.join(gutenberg.raw(file_id) for file_id in gutenberg.fileids())
    webtext_text = ' '.join(webtext.raw(file_id) for file_id in webtext.fileids())
    
    combined_text = (
        reuters_text +
        ' ' + brown_text +
        ' ' + reuters_text +
        ' ' + webtext_text +
        ' ' + brown_text +
        ' ' + reuters_text
    )

    preprocessed_text = preprocess_text(combined_text)
    
    token_count = len(preprocessed_text.split())
    print(f"Total tokens in combined corpus: {token_count}")
    
    return preprocessed_text

def get_training_and_test_data(test_ratio=0.1, random_seed=42):
    import random
    random.seed(random_seed)
    combined_text = get_training_data()
    all_tokens = combined_text.split()
    vocabulary = set(all_tokens)
    vocab_size = len(vocabulary)
    print(f"Vocabulary size: {vocab_size}")
    random.shuffle(all_tokens)
    split_idx = int(len(all_tokens) * (1 - test_ratio))
    train_tokens = all_tokens[:split_idx]
    test_tokens = all_tokens[split_idx:]
    print(f"Training set: {len(train_tokens)} tokens")
    print(f"Test set: {len(test_tokens)} tokens")
    return train_tokens, test_tokens, vocabulary


In [4]:
def tokenize_text(text: str) -> List[str]:
    return word_tokenize(text)

def create_ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    return list(ngrams(tokens, n))

def count_ngrams(ngram_list: List[Tuple[str, ...]]) -> Dict[Tuple[str, ...], int]:
    return Counter(ngram_list)

def get_vocabulary(tokens: List[str]) -> Set[str]:
    return set(tokens)

In [5]:
#2.b Write a function to calculate the probability of a word following a given (n − 1)- gram
def calculate_probability(
    word: str,
    context: Tuple[str, ...], #context here is the n-1 gram.
    ngram_counts: Dict[Tuple[str, ...], int],
    context_counts: Dict[Tuple[str, ...], int],
    vocabulary_size: int,
    smoothing: str = 'laplace'
) -> float:
    

    ngram = context + (word,)
    
    # 2.e Implement smoothing techniques (like Laplace smoothing) to handle the issues of zero probabilities for unseen n-grams.
    if smoothing == 'none':
        # No smoothing - just return relative frequency
        if context in context_counts and context_counts[context] > 0:
            return ngram_counts.get(ngram, 0) / context_counts[context]
        else:
            return 0.0

    elif smoothing == 'laplace':
        # Laplace (add-1) smoothing
        numerator = ngram_counts.get(ngram, 0) + 1
        denominator = context_counts.get(context, 0) + vocabulary_size
        return numerator / denominator

    elif smoothing == 'add_k':
        # Add-k smoothing
        numerator = ngram_counts.get(ngram, 0) + alpha
        denominator = context_counts.get(context, 0) + alpha * vocabulary_size
        return numerator / denominator

    elif smoothing == 'katz':
        # Simple Katz backoff (discounting not implemented, just backoff)
        if ngram_counts.get(ngram, 0) > 0 and context_counts.get(context, 0) > 0:
            return ngram_counts[ngram] / context_counts[context]
        else:
            # Backoff to lower order (unigram)
            return 1 / vocabulary_size

    elif smoothing == 'kneser_ney':
        # Simple absolute discounting Kneser-Ney (D=0.75)
        D = 0.75
        count = ngram_counts.get(ngram, 0)
        context_count = context_counts.get(context, 0)
        if context_count > 0:
            discounted = max(count - D, 0) / context_count
            # Continuation probability: number of unique contexts this word follows / total number of contexts
            continuation_count = sum(1 for ng in ngram_counts if ng[-1] == word)
            total_contexts = len(context_counts)
            lambda_weight = (D / context_count) * len([ng for ng in ngram_counts if ng[:-1] == context])
            p_continuation = continuation_count / total_contexts if total_contexts > 0 else 1 / vocabulary_size
            return discounted + lambda_weight * p_continuation
        else:
            # If context unseen, use uniform probability
            return 1 / vocabulary_size
    else:
        raise ValueError(f"Unknown smoothing method: {smoothing}")


In [6]:
# 2.c Write a function to predict the next word given a sequence of words based on these probabilities.
def predict_next_word(
    context: Tuple[str, ...],
    vocabulary: Set[str],
    ngram_counts: Dict[Tuple[str, ...], int],
    context_counts: Dict[Tuple[str, ...], int],
    n: int,
    smoothing: str = 'laplace',
    k: int = 5
) -> List[Tuple[str, float]]:
    """
    Predict the next word given a context.
    
    Args:
        context: Tuple of preceding words
        vocabulary: Set of all possible words
        ngram_counts: Dictionary of n-gram counts
        context_counts: Dictionary of (n-1)-gram counts
        n: The n in n-gram
        smoothing: Smoothing technique
        k: Number of top predictions to return
        
    Returns:
        List of (word, probability) tuples for the k most likely next words
    """

    if len(context) < n - 1:
        context = ('the',) * ((n - 1) - len(context)) + tuple(context)
    else:
        context = tuple(context[-(n-1):])
    
    vocabulary_size = len(vocabulary)
    word_probs = []
    
    for word in vocabulary:
        prob = calculate_probability(
            word, context, ngram_counts, context_counts, 
            vocabulary_size, smoothing)
        word_probs.append((word, prob))
    return sorted(word_probs, key=lambda x: x[1], reverse=True)[:k]

In [7]:
def predict_next_word_main( vocabulary, ngram_counts, context_counts,n):
    context = ("I", "would")
    smoothing = 'laplace'
    print(f"Context: {context}")
    print("Next words suggestions:")

    for word, prob in predict_next_word(context, vocabulary, ngram_counts, context_counts,n, smoothing, k=5):
        print(f"  {word}: {prob:.6f}")
        


In [8]:
#2.d Write a function to generate a sentence of a specified length given a prefix of (n−1)(or less) words

def generate_sentence(
    length: int,
    vocabulary: Set[str],
    ngram_counts: Dict[Tuple[str, ...], int],
    context_counts: Dict[Tuple[str, ...], int],
    n: int,
    prefix: Optional[List[str]] = None,
    smoothing: str = 'laplace'
) -> List[str]:
    """
    Generate a sentence of specified length given an optional prefix.
    
    Args:
        length: Target length of the sentence in words
        vocabulary: Set of all possible words
        ngram_counts: Dictionary of n-gram counts
        context_counts: Dictionary of (n-1)-gram counts
        n: The n in n-gram
        prefix: Optional list of starting words
        smoothing: Smoothing technique
        alpha: Smoothing parameter for add-k smoothing
        
    Returns:
        Generated sentence as a list of words
    """
    if prefix is None:
        prefix = random.choice(list(context_counts.keys()))
    else:
        if len(prefix) < n - 1:
            prefix = ('the',) * ((n - 1) - len(prefix)) + tuple(prefix)
        else:
            prefix = tuple(prefix[-(n-1):])
    
    sentence = list(prefix)
    
    for _ in range(length - len(prefix)):
        context = tuple(sentence[-(n-1):])
        next_word_candidates = predict_next_word(
            context, vocabulary, ngram_counts, context_counts, 
            n, smoothing
        )
        if not next_word_candidates or next_word_candidates[0][1] == 0:
            next_word = random.choice(list(vocabulary))
        else:
            words, probs = zip(*next_word_candidates)
            probs = np.array(probs)
            probs = probs / probs.sum()
            next_word = np.random.choice(words, p=probs)
        sentence.append(next_word)
    
    return sentence


In [9]:
def test_text_generation(
    vocabulary: Set[str],
    ngram_counts: Dict[Tuple[str, ...], int],
    context_counts: Dict[Tuple[str, ...], int],
    n: int,
    test_prefixes: List[List[str]],
    num_words: int = 10,
    smoothing: str = 'laplace'
) -> Dict[str, List[str]]:
    """
    Test the model by generating text from various prefixes.
    
    Args:
        vocabulary: Set of all words in the vocabulary
        ngram_counts: Dictionary of n-gram counts
        context_counts: Dictionary of (n-1)-gram counts
        n: Size of n-grams
        test_prefixes: List of prefix lists to test (each prefix should have at most n-1 words)
        num_words: Number of words to generate after each prefix
        smoothing: Smoothing technique ('none', 'laplace', or 'good_turing')
        
    Returns:
        Dictionary mapping prefix strings to generated text lists
    """
    results = {}
    
    for prefix in test_prefixes:
        # Ensure prefix is not longer than n-1 words
        if len(prefix) >= n:
            prefix = prefix[-(n-1):]
            
        # Generate text using the prefix
        generated_text = generate_sentence(
            length=num_words,
            vocabulary=vocabulary,
            ngram_counts=ngram_counts,
            context_counts=context_counts,
            n=n,
            prefix=prefix,
            smoothing=smoothing
        )
        
        # Store result with prefix as key
        prefix_str = ' '.join(prefix)
        results[prefix_str] = generated_text
        
        # Print the generated text
        print(f"\nPrefix: '{prefix_str}'")
        print(f"Generated: '{' '.join(generated_text)}'")
    
    return results



In [10]:
def calculate_perplexity(test_tokens, ngram_counts, context_counts, vocabulary_size, n, smoothing='laplace'):

    test_ngrams = create_ngrams(test_tokens, n)
    total_log_prob = 0.0

    unseen_ngrams = 0
    unseen_contexts = 0

    for ngram in test_ngrams:
        context = ngram[:-1]
        ngram_count = ngram_counts.get(ngram, 0)
        context_count = context_counts.get(context, 0)
        if ngram_count == 0:
            unseen_ngrams += 1
        if context_count == 0:
            unseen_contexts += 1
        if smoothing == 'none':
            if context_count > 0 and ngram_count > 0:
                prob = ngram_count / context_count
            else:
                prob = 1 / vocabulary_size
        elif smoothing == 'laplace':
            prob = (ngram_count + alpha) / (context_count + alpha * vocabulary_size)
        else:
            prob = (ngram_count + alpha) / (context_count + alpha * vocabulary_size)
        prob = max(prob, 1e-10)
        total_log_prob += math.log2(prob)

    avg_log_prob = total_log_prob / len(test_ngrams)
    perplexity = 2 ** (-avg_log_prob)

    return perplexity


In [11]:

def compare_ngram_models(
    train_tokens: List[str],
    test_tokens: List[str],
    n_values: List[int],
    smoothing: str = 'laplace'
) -> Dict[int, float]:
    """
    Compare performance of n-gram models with different n values.
    
    Args:
        train_tokens: List of tokens from training set
        test_tokens: List of tokens from test set
        n_values: List of n values to test
        smoothing: Smoothing technique ('none', 'laplace', or 'good_turing')
        alpha: Smoothing parameter for Laplace smoothing
        
    Returns:
        Dictionary mapping n values to perplexity scores
    """
    results = {}
    vocabulary = get_vocabulary(train_tokens)
    vocabulary_size = len(vocabulary)
    
    print(f"\nComparing n-gram models:")
    print(f"{'n':<5}{'Perplexity':<15}")
    print("-" * 20)
    
    for n in n_values:
        # Create and count n-grams for the training set
        ngram_list = create_ngrams(train_tokens, n)
        ngram_counts = count_ngrams(ngram_list)
        
        # Create and count (n-1)-grams for context
        context_list = create_ngrams(train_tokens, n-1)
        context_counts = count_ngrams(context_list)
        
        # Calculate perplexity on test set
        perplexity = calculate_perplexity(
            test_tokens=test_tokens,
            ngram_counts=ngram_counts,
            context_counts=context_counts,
            vocabulary_size=vocabulary_size,
            n=n,
            smoothing=smoothing
        )
        
        results[n] = perplexity
        print(f"{n:<5}{perplexity:<15.4f}")
    
    # Find the best model (lowest perplexity)
    best_n = min(results.items(), key=lambda x: x[1])[0]
    print(f"\nBest model: n={best_n} with perplexity={results[best_n]:.4f}")
    
    return results

In [12]:

def split_data(tokens: List[str], train_ratio: float = 0.8) -> Tuple[List[str], List[str]]:
    """
    Split tokens into training and testing sets.
    
    Args:
        tokens: List of all tokens
        train_ratio: Ratio of tokens to use for training
        
    Returns:
        Tuple of (training_tokens, testing_tokens)
    """
    split_idx = int(len(tokens) * train_ratio)
    return tokens[:split_idx], tokens[split_idx:]

In [13]:
def test_nGram_generation(n, train_tokens, vocabulary, smoothing='laplace'):
    # Test the model with various prefixes for different n-gram sizes
    print("\n--- Testing Text Generation with Various Prefixes for Different n-gram Sizes ---")
    ngram_prefixes = {
        2: [
            ["the"],
            ["in"],
            ["it"],
            ["they"],
            ["she"]
        ],
        3: [
            ["the", "market"],
            ["in", "the"],
            ["it", "is"],
            ["they", "were"],
            ["she", "was"]
        ],
        4: [
            ["the", "stock", "market"],
            ["in", "the", "city"],
            ["it", "is", "a"],
            ["they", "were", "not"],
            ["she", "was", "very"]
        ],
        5: [
            ["the", "stock", "market", "is"],
            ["in", "the", "middle", "of"],
            ["it", "is", "a", "good"],
            ["they", "were", "not", "able"],
            ["she", "was", "very", "happy"]
        ]
    }

    def test_ngram_generation(n, train_tokens, vocabulary, test_prefixes, num_words=8, smoothing='laplace'):
        ngram_list = create_ngrams(train_tokens, n)
        ngram_counts = count_ngrams(ngram_list)
        context_list = create_ngrams(train_tokens, n-1)
        context_counts = count_ngrams(context_list)
        print(f"\n--- {n}-gram Model Text Generation ---")
        for prefix in test_prefixes:
            sentence = generate_sentence(
                length=num_words,
                vocabulary=vocabulary,
                ngram_counts=ngram_counts,
                context_counts=context_counts,
                n=n,
                prefix=prefix,
                smoothing=smoothing
            )
            print(f"Prefix {prefix}: {' '.join(sentence)}")

    for n in range(2, 6):
        test_ngram_generation(
            n=n,
            train_tokens=train_tokens,
            vocabulary=vocabulary,
            test_prefixes=ngram_prefixes[n],
            num_words=8,
            smoothing='laplace'
        )
    


In [14]:
def diagnose_language_model(tokens, n):
    """Analyze the n-gram model statistics for diagnostic purposes"""
    from collections import Counter
    import numpy as np
    
    print(f"Total tokens: {len(tokens)}")
    print(f"Unique tokens (vocabulary size): {len(set(tokens))}")
    
    # Create and count n-grams and contexts
    ngrams = list(zip(*[tokens[i:] for i in range(n)]))
    contexts = list(zip(*[tokens[i:] for i in range(n-1)]))
    
    ngram_counts = Counter(ngrams)
    context_counts = Counter(contexts)
    
    print(f"Total {n}-grams: {len(ngrams)}")
    print(f"Unique {n}-grams: {len(ngram_counts)}")
    print(f"Unique contexts ({n-1}-grams): {len(context_counts)}")
    
    # Analyze n-gram counts
    counts = list(ngram_counts.values())
    print(f"Min n-gram count: {min(counts)}")
    print(f"Max n-gram count: {max(counts)}")
    print(f"Mean n-gram count: {np.mean(counts):.2f}")
    print(f"N-grams appearing once: {counts.count(1)} ({counts.count(1)/len(counts)*100:.2f}%)")
    
    # Check a few probability calculations
    print("\nSample probability calculations:")
    for i, ngram in enumerate(list(ngram_counts.keys())[:3]):
        context = ngram[:-1]
        word = ngram[-1]
        
        ngram_count = ngram_counts[ngram]
        context_count = context_counts[context]
        
        # Calculate probabilities
        mle_prob = ngram_count / context_count
        laplace_prob_a1 = (ngram_count + 1) / (context_count + len(set(tokens)))
        laplace_prob_a01 = (ngram_count + 0.1) / (context_count + 0.1 * len(set(tokens)))
        
        print(f"N-gram {i+1}: {ngram}")
        print(f"  Count: {ngram_count}, Context count: {context_count}")
        print(f"  MLE probability: {mle_prob:.10f}")
        print(f"  Laplace (α=1) probability: {laplace_prob_a1:.10f}")
        print(f"  Laplace (α=0.1) probability: {laplace_prob_a01:.10f}")
    
    return ngram_counts, context_counts



In [15]:
def test_perplexity_for_ngram_range(vocabulary: Set[str], test_tokens: List[str], train_tokens: List[str]):
    """
    Calculate and print perplexity for n-gram models with n from 2 to 5.
    """
    vocabulary_size = len(vocabulary)
    for n in range(2, 6):
        ngram_list = create_ngrams(train_tokens, n)
        ngram_counts = count_ngrams(ngram_list)
        context_list = create_ngrams(train_tokens, n-1)
        context_counts = count_ngrams(context_list)
        perplexity = calculate_perplexity(
            test_tokens=test_tokens,
            ngram_counts=ngram_counts,
            context_counts=context_counts,
            vocabulary_size=vocabulary_size,
            n=n,
            smoothing='laplace'
        )
        print(f"Perplexity for n={n}: {perplexity:.4f}")


In [16]:
def main():

    # Prepare the training data
    raw_text = get_training_data()
    cleaned_text = preprocess_text(raw_text)
    print(f"\nSample of cleaned text: {cleaned_text[:500]}")
    tokens = tokenize_text(cleaned_text)
    print(f"Total tokens: {len(tokens)}")
    train_tokens, test_tokens = split_data(tokens, train_ratio=0.8)
    print(f"Training tokens: {len(train_tokens)}")
    print(f"Testing tokens: {len(test_tokens)}")
    vocabulary = get_vocabulary(train_tokens)
    vocabulary_size = len(vocabulary)
    print(f"Vocabulary size: {vocabulary_size}")
    
    n = 2

    # 2.a. Create n-grams from the tokenized text and calculate their frequencies in the dataset.
    print(f"\n 2.A. Sucessfully created n-grams from the tokenized text and calculated their frequencies in the dataset.")
    ngram_list = create_ngrams(train_tokens, n)
    ngram_counts = count_ngrams(ngram_list)
    context_list = create_ngrams(train_tokens, n-1)
    context_counts = count_ngrams(context_list)

    # 2.b. Calculate the probability of a word following a given (n − 1)-gram.
    print(f"\n 2.B")
    probability = calculate_probability(
        word="market",
        context=("the",),
        ngram_counts=ngram_counts,
        context_counts=context_counts,
        vocabulary_size=vocabulary_size,
        smoothing='laplace'
    )
    print(f"Probability of 'market' following 'the': {probability:.6f}")
    
    # 2.c. Write a function to predict the next word given a sequence of words based on these probabilities.
    print(f"\n 2.C")
    predict_next_word_main( vocabulary, ngram_counts, context_counts,n)

    # 2.d. Predict the next word given a sequence of words based on these probabilities. 
    print(f"\n 2.D")
    sentence = generate_sentence( length=10, vocabulary=vocabulary, ngram_counts=ngram_counts, context_counts=context_counts, n=n, prefix=["the", "market"], smoothing='none')
    print(f"Generated sentence: {' '.join(sentence)}")

    # 2.e. Implement smoothing techniques (like Laplace smoothing) to handle the issues of zero probabilities for unseen n-grams.
    print(f"\n 2.E")
    sentence = generate_sentence( length=10, vocabulary=vocabulary, ngram_counts=ngram_counts, context_counts=context_counts, n=n, prefix=["the", "market"], smoothing='laplace')
    print(f"Generated sentence (With Smoothing Implementation): {' '.join(sentence)}")


    #3.a Test the model by inputting various prefixes (with at most n - 1 words) and evaluating its ability to generate text.
    print(f"\n 3.A")
    test_nGram_generation(n, train_tokens, vocabulary, smoothing='laplace')

    #3.b Compute the perplexity of the model on a test set that was not used during training.
    print("\n 3.B. Perplexity Evaluation")
    perplexity = calculate_perplexity(
        test_tokens=test_tokens,
        ngram_counts=ngram_counts,
        context_counts=context_counts,
        vocabulary_size=vocabulary_size,
        n=n,
        smoothing='none'
    )
    print(f"Perplexity for n={n}: {perplexity:.4f}")

    #3.c Compare the performance of models with different values of n (e.g., bigrams vs. trigrams vs. 4-grams). 
    # Discuss which model achieves lower perplexity and provide insights into why certain n values might be more effective in various contexts.
    print("\nPerplexity for n-grams from 2 to 5:")
    test_perplexity_for_ngram_range(vocabulary, test_tokens, train_tokens)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Gathering texts from multiple corpora...
Total tokens in combined corpus: 6195257

Sample of cleaned text: asian exporters fear damage from usjapan rift mounting trade friction between the us and japan has raised fears among many of asias exporting nations that the row could inflict farreaching economic damage businessmen and officials said they told reuter correspondents in asian capitals a us move against japan might boost protectionist sentiment in the us and lead to curbs on american imports of their products but some exporters said that while the conflict would hurt them in the longrun in the sh
Total tokens: 6196290
Training tokens: 4957032
Testing tokens: 1239258
Vocabulary size: 106731

 2.A. Sucessfully created n-grams from the tokenized text and calculated their frequencies in the dataset.

 2.B
Probability of 'market' following 'the': 0.004563

 2.C
Context: ('I', 'would')
Next words suggestions:
  be: 0.020260
  not: 0.006319
  have: 0.005581
  also: 0.001526
  take: 0.0013